In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split

In [3]:
# Import data
data_new = pd.read_pickle(r'data\04_fct\fct_demographic_offers_and_transactions.pkl')

In [4]:
def process_data(data, gender_column):
    # Filter data for the specified gender
    gender_data = data[data[gender_column] == 1]
    # Extract features and target variable
    features = gender_data[['mobile', 'social', 'web']]
    target = gender_data['offer_viewed']
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)
    return X_train, X_test, y_train, y_test

def train_evaluate_model(X_train, X_test, y_train, y_test):
    # Initialize and train the random forest classifier
    rf_clf = RandomForestClassifier(random_state=42)
    rf_clf.fit(X_train, y_train)
    # Make predictions
    y_pred = rf_clf.predict(X_test)
    y_prob = rf_clf.predict_proba(X_test)[:, 1]
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    class_report = classification_report(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_prob)
    # Calculate permutation importance
    perm_importance = permutation_importance(rf_clf, X_test, y_test, n_repeats=10, random_state=42)
    perm_importance_df = pd.DataFrame({'Feature': ['mobile', 'social', 'web'],
                                       'Importance': perm_importance.importances_mean,
                                       'Importance STD': perm_importance.importances_std})
    return {
        "Accuracy": accuracy,
        "Confusion Matrix": conf_matrix,
        "Classification Report": class_report,
        "ROC-AUC Score": roc_auc,
        "Permutation Importance": perm_importance_df
    }

In [5]:
# Process data for female and male customers
X_train_female, X_test_female, y_train_female, y_test_female = process_data(data_new, 'gender_F')
X_train_male, X_test_male, y_train_male, y_test_male = process_data(data_new, 'gender_M')

# Train and evaluate models for female and male customers
results_female = train_evaluate_model(X_train_female, X_test_female, y_train_female, y_test_female)
results_male = train_evaluate_model(X_train_male, X_test_male, y_train_male, y_test_male)

In [9]:
### Parse model's evaluation results ###
# Initialize the data dictionary
data = {
    "Metric": [],
    "Female": [],
    "Male": []
}

# Add accuracy and ROC-AUC Score
data["Metric"].append("Accuracy")
data["Female"].append(results_female["Accuracy"])
data["Male"].append(results_male["Accuracy"])

data["Metric"].append("ROC-AUC Score")
data["Female"].append(results_female["ROC-AUC Score"])
data["Male"].append(results_male["ROC-AUC Score"])

# Add confusion matrix
data["Metric"].extend(["Confusion Matrix TN", "Confusion Matrix FP", "Confusion Matrix FN", "Confusion Matrix TP"])
data["Female"].extend(results_female["Confusion Matrix"].flatten().tolist())
data["Male"].extend(results_male["Confusion Matrix"].flatten().tolist())

# Function to extract classification report values
def extract_classification_report(report):
    lines = report.split('\n')
    metrics = []
    for line in lines[2:4]:
        metrics.extend(line.split()[1:4])
    return metrics

# Extract classification report details
classification_report_female = extract_classification_report(results_female['Classification Report'])
classification_report_male = extract_classification_report(results_male['Classification Report'])

# Add classification report metrics
classification_metrics = [
    "Precision (0)", "Recall (0)", "F1-Score (0)",
    "Precision (1)", "Recall (1)", "F1-Score (1)"
]

for i, metric_name in enumerate(classification_metrics):
    data["Metric"].append(metric_name)
    data["Female"].append(classification_report_female[i])
    data["Male"].append(classification_report_male[i])

# Add permutation importance
for index, feature in enumerate(results_female['Permutation Importance']['Feature']):
    data["Metric"].append(f'Importance of {feature}')
    data["Female"].append(results_female['Permutation Importance']['Importance'][index])
    data["Male"].append(results_male['Permutation Importance']['Importance'][index])
    
    data["Metric"].append(f'Importance STD of {feature}')
    data["Female"].append(results_female['Permutation Importance']['Importance STD'][index])
    data["Male"].append(results_male['Permutation Importance']['Importance STD'][index])

# Create the DataFrame
df = pd.DataFrame(data)

# Calculate the percent difference
# Convert 'Female' and 'Male' columns to numeric (float) to ensure calculations can be performed
df['Female'] = pd.to_numeric(df['Female'], errors='coerce')
df['Male'] = pd.to_numeric(df['Male'], errors='coerce')

# Calculate percent difference
condition_both_non_zero = (df['Female'] != 0) & (df['Male'] != 0)
condition_one_zero = (df['Female'] == 0) | (df['Male'] == 0)
condition_both_zero = (df['Female'] == 0) & (df['Male'] == 0)

# Calculate percent difference
df['Percent Difference'] = np.nan
df.loc[condition_both_non_zero, 'Percent Difference'] = ((df['Female'] - df['Male']) / ((df['Female'] + df['Male']) / 2)) * 100
df.loc[condition_one_zero & ~condition_both_zero, 'Percent Difference'] = np.nan

df.to_csv(r'data/04_fct/fct_offer_channel_importance_evaluation_results')
df

,Metric,Female,Male,Percent Difference
0,Accuracy,0.798626,0.793343,0.663727
1,ROC-AUC Score,0.811208,0.840370,-3.531331
2,Confusion Matrix TN,427.000000,633.000000,-38.867925
3,Confusion Matrix FP,1115.000000,1653.000000,-38.872832
4,Confusion Matrix FN,263.000000,309.000000,-16.083916
5,Confusion Matrix TP,5038.000000,6899.000000,-31.180364
6,Precision (0),0.620000,0.670000,-7.751938
7,Recall (0),0.280000,0.280000,0.000000
8,F1-Score (0),0.380000,0.390000,-2.597403
9,Precision (1),0.820000,0.810000,1.226994


In this experiment, we used a Random Forest Classifier to see if we could predict if an offer would be completed based off of the offer channels. Based upon the results, the model does a good job predicting the positive cases (F1 = .88), but not the negative cases (F1 = .38). Out of the all the marketing channels, the mobile channel is most important in determining if an offer will be completed (.029).

Conclusion
* Mobile Channel: More important for both genders but slightly more important for males.
* Social Channel: More important for males than females.
* Web Channel: Shows some importance for females but no importance for males.

### Analysis

1. **Accuracy**:
 - The accuracy is similar for both genders, with females at 79.86% and males at 79.33%.

2. **Confusion Matrix**:
 - Both genders show a higher number of false positives (1115 for females and 1653 for males) compared to false negatives (263 for females and 309 for males).
 - The model performs well in identifying true positives for both genders.

3. **Classification Report**:
 - **Precision**: Slightly higher for males (0.81) compared to females (0.82) for the positive class.
 - **Recall**: Higher for both genders for the positive class, with males at 0.96 and females at 0.95.
 - **F1-Score**: Consistently higher for the positive class for both genders.

4. **ROC-AUC Score**:
 - The ROC-AUC score is higher for males (0.8404) compared to females (0.8112), indicating better performance in distinguishing between classes for male customers.

### Conclusion

- The model performs similarly for both genders, with slight differences in precision, recall, and ROC-AUC scores.
- The mobile and social channels are the most important for both genders, with the mobile channel being slightly more important for males.
- The web channel has limited importance overall, with some importance for females but none for males.
